# Try out framework with simulated data

In [ ]:
# TODO: delete this notebook (deprecated)

In [ ]:
import os

import pandas as pd

from q2_ritme.config import (
    experiment_tag,
    host_id,
    ls_model_types,
    mlflow_tracking_uri,
    path_to_ft,
    path_to_md,
    seed_data,
    seed_model,
    target,
    train_size,
)
from q2_ritme.evaluate_all_experiments import (
    best_trial_name,
    compare_trials,
    get_all_exp_analyses,
)
from q2_ritme.evaluate_models import (
    aggregate_best_models_metrics_and_configs,
    get_predictions,
    plot_best_models_comparison,
    plot_model_training_over_iterations,
    plot_rmse_over_experiments,
    plot_rmse_over_time,
    retrieve_best_models,
)
from q2_ritme.process_data import load_n_split_data
from q2_ritme.tune_models import run_all_trials

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load and split data

In [ ]:
train_val, test = load_n_split_data(
    path_to_md, path_to_ft, host_id, target, train_size, seed_data
)

## Run all experiments

In [ ]:
exp_comparison_output = os.path.join("best_models", experiment_tag)

if os.path.exists(exp_comparison_output):
    raise ValueError(
        f"This experiment tag already exists: {experiment_tag}. Please use another one."
    )

In [ ]:
result_dic = run_all_trials(
    train_val,
    target,
    host_id,
    seed_data,
    seed_model,
    mlflow_tracking_uri,
    experiment_tag,
    model_types=ls_model_types,
    fully_reproducible=False,
)

## Evaluate best models: train_val vs. test - performance

In [ ]:
best_model_dic = retrieve_best_models(result_dic)

# Assuming best_model_dic and the datasets are available
non_features = [target, host_id]
features = [x for x in train_val if x not in non_features]

preds_dic = {}
for model_type, tmodel in best_model_dic.items():
    train_pred = get_predictions(train_val, tmodel, target, features, "train")
    test_pred = get_predictions(test, tmodel, target, features, "test")
    all_pred = pd.concat([train_pred, test_pred])

    # save all predictions to model file
    path2save = os.path.join(tmodel.path, "predictions.csv")
    all_pred.to_csv(path2save, index=True)
    preds_dic[model_type] = all_pred

plot_rmse_over_experiments(preds_dic, exp_comparison_output)

In [ ]:
plot_rmse_over_time(preds_dic, ls_model_types, exp_comparison_output)

## Evaluate best models: train vs. val - performance and config

In [ ]:
metrics_all, best_configs = aggregate_best_models_metrics_and_configs(result_dic)

plot_best_models_comparison(metrics_all, exp_comparison_output)

# save best_configs to file
best_configs.to_csv(os.path.join(exp_comparison_output, "config.csv"), index=True)

## Evaluate one model over iterations

In [ ]:
plot_model_training_over_iterations("xgb", result_dic, labels=["data_transform"])

## Postrun evaluation over all experiments performed
experiment > trial

In [ ]:
overall_comparison_output = os.path.join("models", "compare_all")

In [ ]:
# find best trial over all experiments for each model type
best_trials_overall = {}
for model in ls_model_types:
    # read all ExperimentAnalysis objects from this directory
    experiment_dir = f"best_models/*/{model}"
    analyses_ls = get_all_exp_analyses(experiment_dir)

    # identify best trial from all analyses of this model type
    best_trials_overall[model] = best_trial_name(analyses_ls, "rmse_val", mode="min")

compare_trials(best_trials_overall, overall_comparison_output)